In [1]:
from fastai.text.all import *
import polars as pl

In [2]:
pl.Config.set_fmt_str_lengths(200);

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data

In [4]:
labels_dict = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
df = (
    pl.concat([
        pl.scan_csv('emotions-dataset/training.csv').with_columns(is_valid=False),
        pl.scan_csv('emotions-dataset/validation.csv').with_columns(is_valid=True),
    ])
    .with_columns(pl.col.label.replace_strict(labels_dict).alias('label_name'))
    .collect()
)
print(df.shape)
df.head()

(18000, 4)


text,label,is_valid,label_name
str,i64,bool,str
"""i didnt feel humiliated""",0,false,"""sadness"""
"""i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake""",0,false,"""sadness"""
"""im grabbing a minute to post i feel greedy wrong""",3,false,"""anger"""
"""i am ever feeling nostalgic about the fireplace i will know that it is still on the property""",2,false,"""love"""
"""i am feeling grouchy""",3,false,"""anger"""


## Part 1 - LLM

In [5]:
dls = TextDataLoaders.from_df(df.to_pandas(), text_col='text', is_lm=True, valid_pct=0.1)

In [6]:
dls.show_batch(max_n=3)

,text,text_
0,xxbos i understand that he was feeling devastated and i sympathize xxbos i like to pull out when i ever i feel like being snobbish about my musical tastes xxbos i had no obligations except the xxunk which i did nt do i already started missing something that would make free time feel more valuable xxbos i am a christian and appreciate the points but i do feel it would be rejected,i understand that he was feeling devastated and i sympathize xxbos i like to pull out when i ever i feel like being snobbish about my musical tastes xxbos i had no obligations except the xxunk which i did nt do i already started missing something that would make free time feel more valuable xxbos i am a christian and appreciate the points but i do feel it would be rejected by
1,get a record deal ill be happy fuck that xxbos i feel like xxunk is more talented player for sure xxbos i discovered this song called bring me flowers by hope and as beautiful as it is i ca nt help to feel melancholy whenever i listen to it xxbos i want him to feel emotional pain xxbos i feel like that s because i didn t grow up in xxunk at,a record deal ill be happy fuck that xxbos i feel like xxunk is more talented player for sure xxbos i discovered this song called bring me flowers by hope and as beautiful as it is i ca nt help to feel melancholy whenever i listen to it xxbos i want him to feel emotional pain xxbos i feel like that s because i didn t grow up in xxunk at school
2,my new blog is being noticed xxbos i do walk on the treadmill i feel really smug all day so it s worth it xxbos i feel really vulnerable with him i tell him too much i m too honest and i hate it xxbos i feel its a pathetic way to get sympathy xxbos i ca nt decide how i feel about some of the supporting xxunk particularly the girlfriend and,new blog is being noticed xxbos i do walk on the treadmill i feel really smug all day so it s worth it xxbos i feel really vulnerable with him i tell him too much i m too honest and i hate it xxbos i feel its a pathetic way to get sympathy xxbos i ca nt decide how i feel about some of the supporting xxunk particularly the girlfriend and xxunk


### Model

In [7]:
learn = language_model_learner(dls, AWD_LSTM, metrics=accuracy, wd=0.1).to_fp16()

In [8]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(5600, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(5600, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0-2): 3 x RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=5600, bias=True)
    (output_dp): RNNDropout()
  )
)

### Train

In [9]:
learn.fit_one_cycle(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.654675,4.101852,0.267409,00:06
1,4.313936,4.060058,0.275391,00:05


In [10]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.102238,3.987524,0.288146,00:07
1,4.037131,3.941000,0.291811,00:06
2,3.947164,3.920918,0.295259,00:07
3,3.817636,3.920236,0.295428,00:06
4,3.695354,3.942929,0.297261,00:06
5,3.562039,3.948602,0.296513,00:06
6,3.438330,3.970659,0.299648,00:06
7,3.337083,3.994566,0.296393,00:06
8,3.265997,4.012705,0.296803,00:06
9,3.217697,4.020200,0.295669,00:06


In [11]:
learn.save_encoder('lm_encoder')

In [12]:
df.filter(pl.col.is_valid, pl.col.label_name=='joy').sample()

text,label,is_valid,label_name
str,i64,bool,str
"""i look at your pictures but can not touch or feel although they are gorgeous there are not real""",1,true,"""joy"""


In [13]:
inp = 'I am feeling'
samples = 5
for _ in range(samples):
    res = learn.predict(inp, 30, temperature=0.75, no_bar=True)
    print(f'- {res}\n')

- i am feeling so emotional and sad i feel very honored to be part of the team to represent the team to these three international sports i was feeling hopeless and

- i am feeling kind of jealous i feel that the only thing that i could ever do is keep my faithful reader out of the room to know that there is no

- i am feeling amazed i feel like i am in the most dangerous path i could have i feel lucky to have been able to find that little time back

- i am feeling pleased with myself for not having written a book or writing i feel like i m very fucked up now i was feeling pretty doubtful i feel

- i am feeling positive and a bit stressed i am whispered that i feel like i m suffering from a lot of that but i ca nt help feeling rejected i



## Fine-tuning

In [14]:
dls = TextDataLoaders.from_df(df.to_pandas(), text_col='text', label_col='label_name', valid_col='is_valid')

In [15]:
dls.show_batch(max_n=3)

,text,category
0,xxbos i feel in my bones like nobody cares if i m here nobody cares if i m gone here i am again saying i m feeling so lonely people either say its ok to be alone or just go home it xxunk me and i do nt know why it does nt mean i do nt try i try and try but people just treat me like i m a xxunk,sadness
1,xxbos i guess how this clouds your viewing depends on how you feel about xxunk content personally i do nt really give a shit what a film is saying so much as the way its being said and in this case the film is simply too great to ignore but its a sour note in an otherwise delicious xxunk of xxunk,joy
2,xxbos i was not going to be able to sleep until i knew how it ended and mostly because of another thing which i am not even going to talk about here because it makes me angry all over again and also because i feel horribly neurotic and immature getting upset about it and so we will xxunk over that bit,fear


### Model

In [16]:
learn = text_classifier_learner(dls, AWD_LSTM, metrics=accuracy).to_fp16()

In [17]:
learn.model

SequentialRNN(
  (0): SentenceEncoder(
    (module): AWD_LSTM(
      (encoder): Embedding(5600, 400, padding_idx=1)
      (encoder_dp): EmbeddingDropout(
        (emb): Embedding(5600, 400, padding_idx=1)
      )
      (rnns): ModuleList(
        (0): WeightDropout(
          (module): LSTM(400, 1152, batch_first=True)
        )
        (1): WeightDropout(
          (module): LSTM(1152, 1152, batch_first=True)
        )
        (2): WeightDropout(
          (module): LSTM(1152, 400, batch_first=True)
        )
      )
      (input_dp): RNNDropout()
      (hidden_dps): ModuleList(
        (0-2): 3 x RNNDropout()
      )
    )
  )
  (1): PoolingLinearClassifier(
    (layers): Sequential(
      (0): LinBnDrop(
        (0): BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Dropout(p=0.2, inplace=False)
        (2): Linear(in_features=1200, out_features=50, bias=False)
        (3): ReLU(inplace=True)
      )
      (1): LinBnDrop(
        (0): Bat

In [18]:
learn.load_encoder('lm_encoder')

### Train

In [19]:
learn.fit_one_cycle(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.295620,1.152281,0.562500,00:03
1,1.225644,1.118405,0.576500,00:03
2,1.159735,1.078004,0.588000,00:03


In [20]:
learn.unfreeze()

In [21]:
learn.fit_one_cycle(5, 4e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.335026,0.199685,0.926500,00:07
1,0.188903,0.158911,0.936500,00:07
2,0.163141,0.119712,0.939500,00:07
3,0.133339,0.120369,0.948000,00:07
4,0.108476,0.118126,0.942000,00:07


In [22]:
learn.show_results()

,text,category,category_
0,xxbos i tell myself i do nt open my mouth and say what i really feel because i know i m a xxunk and i m smart enough to know i m a xxunk and i never ever know if what i m doing the choices i make are really what i want or need or even the right thing or if its the disease,joy,joy
1,xxbos i could nt hellip even when it made my heart ache to simply look at you hellip because i loved you so much and i knew you would never return my feelings hellip and i could nt bring myself to hate you for the idiotic xxunk you pulled in the other room either though i do ask that you do nt repeat it,sadness,sadness
2,xxbos i really want to watch it for the obvious romance reasons and i have a feeling like it would be a really funny kind of drama too i can also somewhat relate to the female main character who works with this boss who has a lot of pride and is a bit what you would like to call him as xxunk,surprise,surprise
3,xxbos i feel stressed out all the time i said and then i think about how people say stress causes cancer and i know it isn t true but i can t stop thinking that i need to relax or else my cancer will come back and then i get stressed out because i m stressed and it makes me feel worse,sadness,sadness
4,xxbos i could go on and on right now about what we ve been through this year and what i ve learned what xxunk could do when and such but i wo nt because this would be a book and honestly i m not feeling fabulous today and xxunk has been dealing with a giant cold since thursday and we are wiped,joy,joy
5,xxbos i know my xxunk is stronger than my behaviour over the weekend and i need to focus on the joy and health that all the great food i brought with me gives and how i could ve if i really wanted to indulge indulged in that great stuff i know its not the same but i would feel amazing,joy,joy
6,xxbos i just feel you so so don t be afraid i should hurt even more and pray again so i can find you again the more time passes the more it hurts i need you go back in time just one time forgive my sins if only i could turn things back this pain would be so so sloth,fear,fear
7,xxbos i have been out there over the last few weeks i experienced for the first time a feeling of loving the actual act of running of pushing my daughter in the xxunk of getting xxunk by my wife although this would happen if i was in top shape anyway of having cold air nail you in the face,love,love
8,xxbos i suggest you do though it might be hard cause it is a bit slow at times if you don t feel a bit of a tug at your heart or perhaps feel a tear xxunk in your tear xxunk i will declare that you are heartless and thus should be xxunk from the rest of the world,anger,anger


In [23]:
_,lbls,preds = learn.get_preds(with_decoded=True)

In [24]:
(
    pl.DataFrame({'label':lbls, 'pred':preds})
    .group_by('label')
    .agg(
        pl.len(),
        (pl.col.label==pl.col.pred).mean().alias('accuracy'),
    )
    .with_columns(pl.col.label.map_elements(lambda i: dls.categorize.vocab[i], return_dtype=str))
    .sort('label')
)

label,len,accuracy
str,u32,f64
"""anger""",275,0.952727
"""fear""",212,0.877358
"""joy""",704,0.955966
"""love""",178,0.88764
"""sadness""",550,0.970909
"""surprise""",81,0.876543


In [25]:
interp = Interpretation.from_learner(learn)

In [26]:
interp.plot_top_losses(5)

,input,target,predicted,probability,loss
0,xxbos i had applied for a job and they had assured me that the exams would take place a few months later a week later i went to obtain some more information and they told me that the exams had already taken place,anger,joy,0.9973942041397095,7.643234729766846
1,xxbos i was feeling very bah xxunk coming out of this year s thanksgiving weekend and was not thinking pleasant christmas thoughts about the gift giving guilt trip conspiracy run by the marketing xxunk the xxunk and the whole thing in general,joy,anger,0.9899798631668091,6.306465148925781
2,xxbos i guess i feel betrayed because i admired him so much and for someone to do this to his wife and kids just goes beyond the pale,joy,love,0.9946537017822266,5.644493579864502
3,xxbos i did not even think to put shoes on i walked on the snow and could feel warmth from the divine love xxunk from his spirit,joy,fear,0.9894846081733704,5.5434465408325195
4,xxbos my last xxunk midterm a decent grade,joy,anger,0.9905604720115662,5.52032470703125


In [28]:
learn.predict('Write here some random phrase here', with_input=True)

('xxbos xxmaj write here some random phrase here',
 'joy',
 tensor(2),
 tensor([0.2208, 0.0513, 0.6656, 0.0048, 0.0477, 0.0099]))